In [1]:
from apex import amp
from tqdm import tqdm

In [2]:
import numpy as np

In [3]:
import torch.nn as nn
import torch
import torch.nn.functional as f

In [4]:
file_neg = open('/home/ahmed/Desktop/rt-polaritydata/rt-polaritydata/rt-polarity.neg','r', encoding = "ISO-8859-1")
file_pos = open('/home/ahmed/Desktop/rt-polaritydata/rt-polaritydata/rt-polarity.pos','r', encoding = "ISO-8859-1")

In [5]:
dataset = []
labels = []
with open('/home/ahmed/Desktop/rt-polaritydata/rt-polaritydata/rt-polarity.neg','r', encoding = "ISO-8859-1") as myfile:
      for line in myfile:
            dataset.append(line)
            labels.append(1)

In [6]:
with open('/home/ahmed/Desktop/rt-polaritydata/rt-polaritydata/rt-polarity.pos','r', encoding = "ISO-8859-1") as myfile:
      for line in myfile:
            dataset.append(line)
            labels.append(0)

In [7]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/home/ahmed/Desktop/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [8]:
from sklearn.utils import shuffle
dataset , labels = shuffle(dataset,labels)

In [9]:
train_dataset = dataset[:8000]
test_dataset = dataset[8000:]
train_labels = labels[:8000]
test_labels = labels[8000:]

In [10]:
len(train_dataset)

8000

In [11]:
def batch(dataset,labels):
    batch_size = 512
    num_batches = int(len(dataset) / 512)
    for i in range(num_batches+1):
        yield(dataset[i*batch_size:min(len(dataset),(i+1)*batch_size)],
              labels[i*batch_size:min(len(dataset),(i+1)*batch_size)])

In [12]:
word_list = model.index2word
index_mapping = {}
for i,w in enumerate(word_list):
    index_mapping[w] = i

In [13]:
import re
clean_dataset_train = []
for sent in train_dataset:
    sent = re.sub(r'[\n\r]+',' ',sent)
    sent = re.sub(r'[\'\"\.\-\,]+','',sent)
    clean_dataset_train.append(sent)

In [14]:
clean_dataset_test = []
for sent in test_dataset:
    sent = re.sub(r'[\n\r]+',' ',sent)
    sent = re.sub(r'[\'\"\.\-\,]+','',sent)
    clean_dataset_test.append(sent)

In [15]:
train_indexed_dataset = []
for sent in clean_dataset_train:
    encoded_sent = []
    sent = sent.split(' ')
    for word in sent:
        if word in index_mapping:
            encoded_sent.append(index_mapping[word])
    train_indexed_dataset.append(encoded_sent)

In [16]:
test_indexed_dataset = []
for sent in clean_dataset_test:
    encoded_sent = []
    sent = sent.split(' ')
    for word in sent:
        if word in index_mapping:
            encoded_sent.append(index_mapping[word])
    test_indexed_dataset.append(encoded_sent)

In [17]:
googleNews_vectors = model.vectors
googleNews_vectors = torch.from_numpy(googleNews_vectors)

In [18]:
pad_vector = torch.rand(1,300)
end_vector = torch.rand(1,300)
googleNews_vectors = torch.cat((googleNews_vectors,pad_vector,end_vector),0)

In [19]:
googleNews_vectors.shape

torch.Size([3000002, 300])

In [20]:
train_padded_dataset = []
corresponding_train_labels = []
for sent,label in zip(train_indexed_dataset,train_labels):
    length = len(sent)
    if length <46:
        train_padded_dataset.append([0]+sent+[3000000]*(44-length)+[3000001])
        corresponding_train_labels.append(label)
    elif length == 44:
        train_padded_dataset.append([0]+sent+[3000001])
        corresponding_train_labels.append(label)

In [21]:
test_padded_dataset = []
corresponding_test_labels = []
for sent,label in zip(test_indexed_dataset,test_labels):
    length = len(sent)
    if length <46:
        test_padded_dataset.append([0]+sent+[3000000]*(44-length)+[3000001])
        corresponding_test_labels.append(label)
    elif length == 44:
        test_padded_dataset.append([0]+sent+[3000001])
        corresponding_test_labels.append(label)

In [22]:
device = torch.device('cuda')

In [23]:
X_train = torch.tensor(train_padded_dataset)
X_test = torch.tensor(test_padded_dataset)

In [24]:
print(X_train.shape)
print(X_test.shape)

torch.Size([7999, 46])
torch.Size([2662, 46])


In [24]:
Y_train = torch.tensor(train_labels)
Y_test = torch.tensor(test_labels)

In [25]:
Y_train = Y_train.unsqueeze(-1)

In [26]:
Y_test = Y_test.unsqueeze(-1)

In [27]:
import torch.nn as nn

In [90]:
class CNN_TC(nn.Module):
    def __init__(self,googleNews_vectors,Filters):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(googleNews_vectors,freeze=True)
        self.embedding.weight.requires_grad = False
        self.conv1 = nn.Conv1d(1,1,kernel_size=(Filters[0] * 300),stride=(300))
        self.conv2 = nn.Conv1d(1,1,kernel_size=(Filters[1] * 300),stride=(300))
        self.conv3 = nn.Conv1d(1,1,kernel_size=(Filters[2] * 300),stride=(300))
        
        self.max_conv1 = nn.MaxPool1d(kernel_size=(15))
        self.max_conv2 = nn.MaxPool1d(kernel_size=(11))
        self.max_conv3 = nn.MaxPool1d(kernel_size=(9))
        
        self.FFN1 = nn.Linear(9,100)
        self.FFN2 = nn.Linear(100,2)
        
    def forward(self,inputs):
        X = self.embedding(inputs)
        X = X.view(X.shape[0],1,-1).contiguous()

        max_feature1 = self.max_conv1(f.relu(self.conv1(X)))
        max_feature2 = self.max_conv2(f.relu(self.conv2(X)))
        max_feature3 = self.max_conv3(f.relu(self.conv3(X)))
        features = torch.cat((max_feature1,max_feature2,max_feature3), -1).squeeze(0)
                             
        output = self.FFN2(f.relu(self.FFN1(features)))
        return output

In [85]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [86]:
count_parameters(model)

4805

In [91]:
torch.cuda.empty_cache()

In [ ]:
model = CNN_TC(googleNews_vectors,Filters=[3,4,5]).to(device)

In [ ]:
from apex.optimizers import FusedAdam

In [75]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
model, optimizer = amp.initialize(model, optimizer,opt_level='O3')

In [ ]:
torch.cuda.ca

In [71]:
def accuracy(Y_hat,Y):
    predicted = np.argmax(Y_hat.cpu().data.numpy(),axis=1)
    acc = sum([1 if p==y else 0 for p,y in zip(predicted,Y)]) / len(Y_hat)
    return acc

In [72]:
import time
start = time.time()
epochs = 40

for epoch in range(epochs):
    train_accuracy = 0
    count = 0
    accumulated_loss = 0
    for train_in,train_label in tqdm(batch(X_train,Y_train)):
        #print(train_in)
        torch.cuda.synchronize()
        output = model(train_in.to(device))
        #print(output.shape)
        output = output.squeeze(1)
        #print(output.shape)
        train_label = train_label.squeeze(-1)
        loss = criterion(output,train_label.long().to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        optimizer.step()
        count += 1
        train_accuracy += accuracy(output,train_label)
        accumulated_loss += scaled_loss.detach() #scaled_loss
        optimizer.zero_grad()
       
    print('Training Epoch {} with Accuracy :{} Loss :{}'.format(epoch+1,train_accuracy/count,accumulated_loss/count))
    with torch.no_grad():
        test_acc = 0
        test_count = 0
        for X_t,Y_t in tqdm(batch(X_test,Y_test)):
            test_output = model(X_t.to(device))
            test_output = test_output.squeeze(1)

            Y_t = Y_t.squeeze(-1)
            test_accuracy = accuracy(test_output,Y_t)
            test_acc += test_accuracy
            test_count += 1
            
    print('Testing Epoch {} with Accuracy :{}'.format(epoch+1,test_acc/test_count))  
    
end = time.time()
print("time equal {}".format(end-start))

16it [00:00, 17.11it/s]
3it [00:00, 25.75it/s]

Training Epoch 1 with Accuracy :0.5423583984375 Loss :45285.83984375


6it [00:00, 29.23it/s]
2it [00:00, 16.65it/s]

Testing Epoch 1 with Accuracy :0.5740672700082509


16it [00:00, 17.21it/s]
3it [00:00, 26.05it/s]

...

Testing Epoch 39 with Accuracy :0.654010029909241


16it [00:00, 17.37it/s]
3it [00:00, 25.64it/s]

Training Epoch 40 with Accuracy :0.90380859375 Loss :17174.318359375


6it [00:00, 29.23it/s]

Testing Epoch 40 with Accuracy :0.6629118966584159
time equal 44.58629536628723


In [77]:
torch.cuda.memory_allocated() 

9045737984

In [ ]:
class CNN_TC(nn.Module):
    def __init__(self,googleNews_vectors,Filters):
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(googleNews_vectors,freeze=True)
        self.embedding.weight.requires_grad = False
        self.conv1 = nn.Conv1d(1,1,kernel_size=(Filters[0] * 300),stride=(300))
        self.conv2 = nn.Conv1d(1,1,kernel_size=(Filters[1] * 300),stride=(300))
        self.conv3 = nn.Conv1d(1,1,kernel_size=(Filters[2] * 300),stride=(300))
        
        self.max_conv1 = nn.MaxPool1d(kernel_size=(15))
        self.max_conv2 = nn.MaxPool1d(kernel_size=(11))
        self.max_conv3 = nn.MaxPool1d(kernel_size=(9))
        
        self.FFN1 = nn.Linear(9,20)
        self.FFN2 = nn.Linear(20,2)
        
    def forward(self,inputs):
        X = self.embedding(inputs)
        X = X.view(X.shape[0],1,-1).contiguous()

        max_feature1 = self.max_conv1(f.relu(self.conv1(X)))
        max_feature2 = self.max_conv2(f.relu(self.conv2(X)))
        max_feature3 = self.max_conv3(f.relu(self.conv3(X)))
        features = torch.cat((max_feature1,max_feature2,max_feature3), -1).squeeze(0)
                             
        output = self.FFN2(f.relu(self.FFN1(features)))
        return output
    
    
model = CNN_TC(googleNews_vectors,Filters=[3,4,5]).to(device)
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().cuda()
model, optimizer = amp.initialize(model, optimizer,opt_level='O1')
import time
start = time.time()
epochs = 40

for epoch in range(epochs):
    train_accuracy = 0
    count = 0
    accumulated_loss = 0
    for train_in,train_label in tqdm(batch(X_train,Y_train)):
        #print(train_in)
        torch.cuda.synchronize()
        output = model(train_in.to(device))
        #print(output.shape)
        output = output.squeeze(1)
        #print(output.shape)
        train_label = train_label.squeeze(-1)
        loss = criterion(output,train_label.long().to(device))
        #with amp.scale_loss(loss, optimizer) as scaled_loss:
            #scaled_loss.backward()
        loss.backward()
        optimizer.step()
        count += 1
        train_accuracy += accuracy(output,train_label)
        accumulated_loss += loss#scaled_loss
        optimizer.zero_grad()
       
    print('Training Epoch {} with Accuracy :{} Loss :{}'.format(epoch+1,train_accuracy/count,accumulated_loss/count))
    with torch.no_grad():
        test_acc = 0
        test_count = 0
        for X_t,Y_t in tqdm(batch(X_test,Y_test)):
            test_output = model(X_t.to(device))
            test_output = test_output.squeeze(1)

            Y_t = Y_t.squeeze(-1)
            test_accuracy = accuracy(test_output,Y_t)
            test_acc += test_accuracy
            test_count += 1
            
    print('Testing Epoch {} with Accuracy :{}'.format(epoch+1,test_acc/test_count))  
    
end = time.time()
print("time equal {}".format(end-start))